# Model 2: M/G/2
Based on Lab 5 code

In [1]:
"""(q4.py) M/M/c queueing system with several monitors and multiple replications"""

from SimPy.Simulation import *
import random
import numpy
import math

In [2]:
def conf(L):
    """confidence interval"""
    lower = numpy.mean(L) - 1.96*numpy.std(L)/math.sqrt(len(L))
    upper = numpy.mean(L) + 1.96*numpy.std(L)/math.sqrt(len(L))
    return lower, upper

In [3]:
class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, serve_dist):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(serve_dist))
            t = random.expovariate(lamb)
            yield hold, self, t

In [4]:
class Arrival(Process):
    """an arrival"""
    n = 0  # class variable (number in system)

    def run(self, serve_dist):
        # Event: arrival
        Arrival.n += 1   # number in system
        arrivetime = now()
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)

        yield request, self, G.server
        # ... waiting in queue for server to be empty (delay) ...
      
        # Event: service begins
        t = serve_dist()
      
        yield hold, self, t
        # ... now being served (activity) ...
      
        # Event: service ends
        yield release, self, G.server # let go of server (takes no simulation time)
        Arrival.n -= 1
        G.numbermon.observe(Arrival.n)
        if (Arrival.n>0):
            G.busymon.observe(1)
        else:
            G.busymon.observe(0)
        delay = now()-arrivetime
        G.delaymon.observe(delay)

In [5]:
class G:
    server = 'dummy'
    delaymon = 'Monitor'
    numbermon = 'Monitor'
    busymon = 'Monitor'

In [6]:
def model(c, N, lamb, serve_dist, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    G.server = Resource(c)
    G.delaymon = Monitor()
    G.numbermon = Monitor()
    G.busymon = Monitor()
    
    Arrival.n = 0
   
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, serve_dist))
    simulate(until=maxtime)

    # gather performance measures
    W = G.delaymon.mean()
    L = G.numbermon.timeAverage()
    B = G.busymon.timeAverage()
    return W, L, B

In [24]:
## The serving distribution is described by 0.65*expon.pdf(x, 0, 10) + 0.35*norm.pdf(x, 56, 8)

def serve_dist():
    if random.random() < 0.965:
        return random.expovariate(1/1.5)
    else:
        return random.normalvariate(60, 2)

## Experiment ----------------
allW = []
allL = []
allB = []
allLambdaEffective = []
for k in range(10):
    seed = 123*k
    result = model(c=2, N=10000, lamb=1/2.445357142857143, serve_dist=serve_dist, maxtime=200000, rvseed=seed)
    allW.append(result[0])
    allL.append(result[1])
    allB.append(result[2])
    allLambdaEffective.append(result[1]/result[0])

In [25]:
print("Estimate of W:", numpy.mean(allW))
print("Conf int of W:", conf(allW))
print("Estimate of L:", numpy.mean(allL))
print("Conf int of L:", conf(allL))
print("Estimate of B:", numpy.mean(allB))
print("Conf int of B:", conf(allB))
print("Estimate of LambdaEffective:", numpy.mean(allLambdaEffective))
print("Conf int of LambdaEffective:", conf(allLambdaEffective))

Estimate of W: 23.647644551502466
Conf int of W: (22.100964873584743, 25.19432422942019)
Estimate of L: 9.714075842636483
Conf int of L: (9.08782242360543, 10.340329261667534)
Estimate of B: 0.8322252651171897
Conf int of B: (0.8248918129668595, 0.83955871726752)
Estimate of LambdaEffective: 0.4108958562690838
Conf int of LambdaEffective: (0.4080619525184981, 0.41372976001966955)
